<a href="https://colab.research.google.com/github/rakuri2026/VM0047/blob/main/testing_vcal_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Install required packages
!pip install -q streamlit pandas pyngrok

In [29]:
%%writefile app.py
import streamlit as st
import pandas as pd
import geopandas as gpd
import numpy as np
import re
import math
import os

# --------------------------------------------------------------
# 1. REFERENCE SPECIES TABLE
# --------------------------------------------------------------
species_data = {
    'SN': range(1, 26),
    'scientific_name': ['Abies spp', 'Acacia catechu', 'Adina cardifolia', 'Albizia spp', 'Alnus nepalensis',
                       'Anogeissus latifolia', 'Bombax ceiba', 'Cedrela toona', 'Dalbergia sissoo',
                       'Eugenia Jambolana', 'Hymenodictyon excelsum', 'Lagerstroemia parviflora',
                       'Michelia champaca', 'Pinus roxburghii', 'Pinus wallichiana', 'Quercus spp',
                       'Schima wallichii', 'Shorea robusta', 'Terminalia alata', 'Trewia nudiflora',
                       'Tsuga spp', 'Terai spp', 'Hill spp', 'Coniferious', 'Broadleaved'],
    'a': [-2.4453, -2.3256, -2.5626, -2.4284, -2.7761, -2.272, -2.3856, -2.1832, -2.1959, -2.5693,
          -2.585, -2.3411, -2.0152, -2.977, -2.8195, -2.36, -2.7385, -2.4554, -2.4616, -2.4585,
          -2.5293, -2.3993, -2.3204, None, None],
    'b': [1.722, 1.6476, 1.8598, 1.7609, 1.9006, 1.7499, 1.7414, 1.8679, 1.6567, 1.8816,
          1.9437, 1.7246, 1.8555, 1.9235, 1.725, 1.968, 1.8155, 1.9026, 1.8497, 1.8043,
          1.7815, 1.7836, 1.8507, None, None],
    'c': [1.0757, 1.0552, 0.8783, 0.9662, 0.9428, 0.9174, 1.0063, 0.7569, 0.9899, 0.8498,
          0.7902, 0.9702, 0.763, 1.0019, 1.1623, 0.7496, 1.0072, 0.8352, 0.88, 0.922,
          1.0369, 0.9546, 0.8223, None, None],
    'a1': [5.4433, 5.4401, 5.4681, 4.4031, 6.019, 4.9502, 4.5554, 4.9705, 4.358, 5.1749,
           5.5572, 5.3349, 3.3499, 6.2696, 5.7216, 4.8511, 7.4617, 5.2026, 4.5968, 5.3475,
           5.2774, 4.8991, 5.5323, None, None],
    'b1': [-2.6902, -2.491, -2.491, -2.2094, -2.7271, -2.3353, -2.3009, -2.3436, -2.1559, -2.3636,
           -2.496, -2.4428, -2.0161, -2.8252, -2.6788, -2.4494, -3.0676, -2.4788, -2.2305, -2.4774,
           -2.6483, -2.3406, -2.4815, None, None],
    's': [0.436, 0.443, 0.443, 0.443, 0.803, 0.443, 0.443, 0.443, 0.684, 0.443,
          0.443, 0.443, 0.443, 0.189, 0.683, 0.747, 0.52, 0.055, 0.443, 0.443,
          0.443, 0.443, 0.443, 0.436, 0.443],
    'm': [0.372, 0.511, 0.511, 0.511, 1.226, 0.511, 0.511, 0.511, 0.684, 0.511,
          0.511, 0.511, 0.511, 0.256, 0.488, 0.96, 0.186, 0.341, 0.511, 0.511,
          0.511, 0.511, 0.511, 0.372, 0.511],
    'bg': [0.355, 0.71, 0.71, 0.71, 1.51, 0.71, 0.71, 0.71, 0.684, 0.71,
           0.71, 0.71, 0.71, 0.3, 0.41, 1.06, 0.168, 0.357, 0.71, 0.71,
           0.71, 0.71, 0.71, 0.355, 0.71],
    'Local_Name': ['Thingre Salla', 'Khayar', 'Karma', 'Siris', 'Uttis', 'Banjhi', 'Simal', 'Tooni',
                   'Sissoo', 'Jamun', 'Bhudkul', 'Botdhayero', 'Chanp', 'Khote Salla', 'Gobre Salla',
                   'Kharsu', 'Chilaune', 'Sal', 'Saj', 'Gamhari', 'Dhupi Salla', 'Terai Spp',
                   'Hill spp', None, None]
}
df_ref = pd.DataFrame(species_data)

# --------------------------------------------------------------
# 2. COORDINATE HELPERS
# --------------------------------------------------------------
COORD_PATTERNS = {
    "lon": re.compile(r"^(lon|longitude|x|east|easting|x.?coord)", re.I),
    "lat": re.compile(r"^(lat|latitude|y|north|northing|y.?coord)", re.I),
}
def guess_coord_cols(df):
    lon = lat = None
    for c in df.columns:
        if COORD_PATTERNS["lon"].search(c): lon = c
        elif COORD_PATTERNS["lat"].search(c): lat = c
        if lon and lat: break
    return lon, lat

def guess_is_geographic(x, y, epsg=None):
    if epsg == 4326: return True
    if epsg and epsg != 4326: return False
    return x.between(-200,200).all() and y.between(-100,100).all()

def utm_zone_from_lon(lon):
    zone = int((lon + 180)//6) + 1
    return 32600 + zone

# --------------------------------------------------------------
# 3. CALCULATION FUNCTION
# --------------------------------------------------------------
def add_calculated_columns(df):
    df = df.copy()
    df['stem_volume'] = (df['a'] + df['b'] * df['dia_cm'].apply(lambda x: math.log(x)) + df['c'] * df['height_m'].apply(lambda x: math.log(x))).apply(math.exp) / 1000
    df['branch_ratio'] = df['dia_cm'].apply(lambda x: 0.1 if x < 10 else 0.2)
    df['branch_volume'] = df['stem_volume'] * df['branch_ratio']
    df['tree_volume'] = df['stem_volume'] + df['branch_volume']
    df['cm10diaratio'] = (df['a1'] + df['b1'] * df['dia_cm'].apply(lambda x: math.log(x))).apply(math.exp)
    df['cm10topvolume'] = df['stem_volume'] * df['cm10diaratio']
    df['gross_volume'] = df['stem_volume'] - df['cm10topvolume']
    df['net_volume'] = df.apply(lambda row: row['gross_volume'] * 0.9 if row['class'] == 'A' else row['gross_volume'] * 0.8, axis=1)
    df['net_volum_cft'] = df['net_volume'] * 35.3147
    df['firewood_m3'] = df['tree_volume'] - df['net_volume']
    df['firewood_chatta'] = df['firewood_m3'] * 0.105944
    return df

# --------------------------------------------------------------
# 4. STREAMLIT APP
# --------------------------------------------------------------
st.set_page_config(page_title="Forest Inventory + Volume Calc", layout="wide")
st.title("Forest Inventory: Coordinates + Species + Volume Calculation")

uploaded_file = st.file_uploader("Upload CSV (must have: `dia_cm`, `height_m`, `class`, `species`, coordinates)", type=["csv"])

if uploaded_file is not None:
    try:
        df = pd.read_csv(uploaded_file)
        st.success(f"Loaded {df.shape[0]:,} rows")
    except Exception as e:
        st.error(f"Error: {e}")
        st.stop()

    # --- Coordinate Handling ---
    st.subheader("1. Coordinate Detection & Reprojection")
    auto_lon, auto_lat = guess_coord_cols(df)
    c1, c2 = st.columns(2)
    with c1:
        lon_col = st.selectbox("X/Easting", df.columns, index=df.columns.get_loc(auto_lon) if auto_lon in df.columns else 0)
    with c2:
        lat_col = st.selectbox("Y/Northing", df.columns, index=df.columns.get_loc(auto_lat) if auto_lat in df.columns else 0)

    df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')
    df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
    valid_coords = df[[lon_col, lat_col]].dropna()

    if valid_coords.empty:
        st.error("No valid coordinates.")
        st.stop()

    epsg_in = st.text_input("Source EPSG (optional)", placeholder="32645")
    user_epsg = int(epsg_in) if epsg_in.strip() else None
    is_geo = guess_is_geographic(valid_coords[lon_col], valid_coords[lat_col], user_epsg)

    if user_epsg:
        src_epsg = user_epsg
    elif is_geo:
        src_epsg = 4326
    else:
        approx_lon = (valid_coords[lon_col].median() - 500000) / 111319
        src_epsg = utm_zone_from_lon(approx_lon)

    gdf = gpd.GeoDataFrame(valid_coords, geometry=gpd.points_from_xy(valid_coords[lon_col], valid_coords[lat_col]), crs=f"EPSG:{src_epsg}")
    if src_epsg != 4326:
        gdf = gdf.to_crs("EPSG:4326")
    gdf["longitude"] = gdf.geometry.x
    gdf["latitude"] = gdf.geometry.y

    df_with_coords = df.loc[gdf.index].copy()
    df_with_coords["longitude"] = gdf["longitude"]
    df_with_coords["latitude"] = gdf["latitude"]

    # --- Species Join ---
    st.subheader("2. Species Parameter Join")
    if "species" not in df_with_coords.columns:
        st.error("Column `species` not found. Required for volume calculation.")
        st.stop()

    joined_df = df_with_coords.merge(df_ref, left_on="species", right_on="scientific_name", how="left")
    joined_df = joined_df.drop(columns=["scientific_name"], errors="ignore")

    # --- Volume Calculation ---
    st.subheader("3. Volume & Biomass Calculation")
    required_cols = ['dia_cm', 'height_m', 'class']
    missing = [col for col in required_cols if col not in joined_df.columns]
    if missing:
        st.error(f"Missing columns for calculation: {missing}")
        st.stop()

    result_df = joined_df.copy()
    result_df = add_calculated_columns(result_df)

    st.success(f"Calculated volumes for {len(result_df):,} trees")
    st.dataframe(result_df.head(10))

    # --- Map ---
    st.subheader("4. Map")
    map_df = result_df[["latitude", "longitude"]].rename(columns={"latitude": "lat", "longitude": "lon"})
    st.map(map_df)

    # --- Downloads ---
    st.subheader("5. Export")
    csv_full = result_df.to_csv(index=False).encode()
    st.download_button("Download Full Result (CSV)", csv_full, "forest_inventory_result.csv", "text/csv")

    # Export for notebook
    result_df.to_pickle("result_df.pkl")
    st.success("`result_df.pkl` saved – load in notebook: `pd.read_pickle('result_df.pkl')`")

else:
    st.info("Upload CSV with: `species`, `dia_cm`, `height_m`, `class`, and coordinates.")
    st.code("""Example:
species,dia_cm,height_m,class,easting,northing
Shorea robusta,45,25,A,500000,4649776
Acacia catechu,28,18,B,500100,4649800
""")

Overwriting app.py


#for streamlit brower code

In [30]:
from pyngrok import ngrok
import os
import signal

# Authenticate with ngrok using your authtoken
ngrok.set_auth_token("34hVSy1cfuYmYxlminhoFdP7bqf_TcEDNnxWTqzS6VPBPeCH")

# Kill any running ngrok processes
try:
    ngrok.kill()
except:
    pass # Ignore if no ngrok process is running

# Start Streamlit in the background
!streamlit run app.py --server.port=8501 &>/content/logs.txt &

# Create a public URL using ngrok
public_url = ngrok.connect(8501)
print(f"🚀 Your Streamlit app is live at: {public_url}")

🚀 Your Streamlit app is live at: NgrokTunnel: "https://forkier-verna-lathy.ngrok-free.dev" -> "http://localhost:8501"
